## Lesson 10 - Create Nodes, Relations using  Cypher





### Table of Contents

* [Introduction to Cypher](#IntroductionCypher)
* [Creating Nodes and Relationships in the Graph](#CreatingNodesRelationshipsGraph)
* [Explore Movie Graph](#ImportMovieGraph)
* [Creating Nodes and Relationships in the Graph  with Cypher](#CreatingNodesRelationshipsGraph)
* [Neo4j databases and graphs](#DefaultNeo4jDatasets)
* [Neo4j Sandbox](#Neo4jSandbox)




<a id="IntroductionCypher"></a>
## Introduction to Cypher

Cypher is the query language you use to retrieve data from the Neo4j Database, as well as create and update the data.

At the end of this module, you should be able to write Cypher statements to:

- Retrieve nodes from the graph.
- Filter nodes retrieved using labels and property values of nodes.
- Retrieve property values from nodes in the graph.
- Filter nodes retrieved using relationships.

### What is Cypher?

Cypher is a declarative query language that allows for expressive and efficient querying and updating of graph data. Cypher is a relatively simple and very powerful language. Complex database queries can easily be expressed through Cypher, allowing you to focus on your domain instead of getting lost in the syntax of database access.

Cypher is designed to be a human-friendly query language, suitable for both developers and other professionals. The guiding goal is to make the simple things easy, and the complex things possible.

### Cypher is ASCII art

Optimized for being read by humans, Cypher's construct uses English prose and iconography (called ASCII Art) to make queries more self-explanatory.Cypher is ASCII art

<img src="images/AsciiArt.png" width=80%>

### Cypher to express what to retrieve not how

Being a declarative language, Cypher focuses on the clarity of expressing what to retrieve from a graph, not on how to retrieve it. You can think of Cypher as mapping English language sentence structure to patterns in a graph. For example, the nouns are nodes of the graph, the verbs are the relationships in the graph, and the adjectives and adverbs are the properties.

<img src="images/Nouns.png">

This is in contrast to imperative, programmatic APIs for database access. This approach makes query optimization an implementation detail instead of a burden on the developer, removing the requirement to update all traversals just because the physical database structure has changed.

Cypher is inspired by a number of different approaches and builds upon established practices for expressive querying. Many of the Cypher keywords like [WHERE] and [ORDER BY] are inspired by SQL. The pattern matching functionality of Cypher borrows concepts from SPARQL. And some of the collection semantics have been borrowed from languages such as Haskell and Python.

The Cypher language has been made available to anyone to implement and use via openCypher (opencypher.org), allowing any database vendor, researcher or other interested party to reap the benefits of our years of effort and experience in developing a first class graph query language.

<a id="ImportMovieGraph"></a>
## 匯入電影資料集 (Explore Movie Graph)

於console列中輸入以下命令，並按下右方的三角形按鈕執行

```:play movie-graph```

The Movie Graph is a mini graph application containing actors and directors that are related through the movies they've collaborated on.

This guide will show you how to:

- Create: insert movie data into the graph
- Find: retrieve individual movies and actors
- Query: discover related actors and directors
- Solve: the Bacon Path

<img src="images/MovieGraph01.png">

會跳出電影資料庫的教學，翻到第二頁後點選 CREATE 指令如圖

<img src="images/MovieGraph02.png">

執行後就會產生電影資料庫囉！（※注意：請只執行一次）

Movie DB 是內建的迷你電影資料庫，儲存著許多電影、導演以及演員們的關係圖，是官方作為 Cypher 語法教學與練習的基礎。

如果你想看 Cypher 完全基礎教學，請輸入

```:play cypher```


### 請找出湯姆漢克有演的電影，年份是 1990 ~ 2000，最多列出 10 筆

```
MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies)
WHERE tomHanksMovies.released >= 1990 AND tomHanksMovies.released < 2000
RETURN tom, tomHanksMovies
LIMIT 10
```

<img src="images/MovieGraph03.png">

### 請找出電影「雲圖」的所有導演，而這些導演們還有哪些其他作品？

```
MATCH (cloudAtlas {title: "Cloud Atlas"})<-[:DIRECTED]-(directors)-[:DIRECTED]->(m)
RETURN *
```

<img src="images/MovieGraph04.png">


### 請找出有哪些人曾和湯姆漢克一起演出，分別是哪些電影？

```
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors)
RETURN coActors
```

<img src="images/MovieGraph05.png">

### 請列出所有跟電影「駭客任務 The Matrix」有關的人有誰？分別是什麼樣的關係？

```
MATCH (people:Person)-[relatedTo]-(:Movie {title: "The Matrix"}) RETURN people.name, Type(relatedTo), relatedTo
```

<img src="images/MovieGraph06.png">

上述查詢將關係當做了變數做查詢，所以導演或演員們都會被找出。

接下來我們試著新增自己喜歡的電影、導演和演員們，首先我們看看克里斯多福·諾蘭有沒有存在資料庫中?

```
MATCH (p:Person) WHERE p.name =~ '.*Nolan.*'
RETURN p
```

<img src="images/MovieGraph07.png">

這是以規則運算式來做查詢，只要名字包含了 "Nolan"，就會被找到，就像是 SQL 的 LIKE 語法；做 LIKE 查詢還有一個方法如下：

```
MATCH (p:Person) WHERE p.name CONTAINS 'Nolan'
RETURN p
```

<img src="images/MovieGraph08.png">

查詢結果是，這位大導演還未被加入這個電影資料庫，那我們自己來建立吧！這邊我想改用中文，畢竟在台灣，大部分還是會以中譯來命名。

```
CREATE (nolan:Person {name: '諾蘭'}),
(m1:Movie {title: '星際效應', released: 2014}),
(m2:Movie {title: '全面啟動', released: 2010}),
(m3:Movie {title: 'TENET天能', released: 2020}),
(nolan)-[:DIRECTED]->(m1),
(nolan)-[:DIRECTED]->(m2),
(nolan)-[:DIRECTED]->(m3)
```

<img src="images/MovieGraph09.png">

再執行查詢導演名稱：

```
MATCH (p:Person) WHERE p.name =~ '.*諾蘭.*'
RETURN p
```

<img src="images/MovieGraph10.png">

雖然只有一個點，點擊導演「諾蘭」後，展開與其的關聯，即可看見剛才建立他所導演的電影關聯了。

<img src="images/MovieGraph11.png">

## Neo4j 看圖說故事 - 諾蘭執導的電影

最近有一部很火紅的電影 - 天能，身邊朋友們都說好看，所以為了看懂，大衛也去三刷了，如果用圖來說故事，會是怎樣的世界觀?

#### 新增 Node 前都得先確認是否存在嗎？

傳統資料庫中，若想找諾蘭的電影，會先確認他是否存在資料庫中？沒有的話才新增，但難道每次都得先 MATCH 後 CREATE ? 有沒有更方便的做法？

```
MERGE (nolan:Person {name: '諾蘭'})
RETURN nolan
```

MERGE 會先比對這筆資料是否已存在？沒有的話才新增，否則取既有的資料即可。除了讓語法更簡潔方便，也同時解決可能會重複新增 Node 的問題。

#### 類似的資料或關係，只能不斷 Copy-Paste Cypher敘述嗎?

如每一次新增一個諾蘭的電影，並指定其導演關係，就要重複貼上如下:

```
CREATE (nolan:Person {name: '諾蘭'}),
(m1:Movie {title: '星際效應', released: 2014}),
(m2:Movie {title: '全面啟動', released: 2010}),
(m3:Movie {title: 'TENET天能', released: 2020}),
(nolan)-[:DIRECTED]->(m1),
(nolan)-[:DIRECTED]->(m2),
(nolan)-[:DIRECTED]->(m3)
```

但是他的電影很多啊！上面的語法只是新增其中三個，每次都要重複寫 Movie，有什麼方法可以更簡潔的輸入呢？接下來我們介紹 UNWIND

根據官方文件："With UNWIND, you can transform any list back into individual rows." 請留意 `list`和`individual rows`，說明輸入只要是列表就行，輸出是每一個元素，利用 UNWIND 可以將陣列元素轉換成一筆一筆的資料列，做個簡單的舉例：


```
UNWIND [1, 2, 3, NULL, 5] AS x RETURN x

output:
x
1
2
3
null
5
```

我們將語法修改如下

```
UNWIND [
    {title: '追隨', released: 1998},
    {title: '記憶拼圖', released: 2000},
    {title: '針鋒相對', released: 2002},
    {title: '蝙蝠俠：開戰時刻', released: 2005},
    {title: '頂尖對決', released: 2006},
    {title: '黑暗騎士', released: 2008},
    {title: '全面啟動', released: 2010},
    {title: '黑暗騎士：黎明昇起', released: 2012},
    {title: '星際效應', released: 2014},
    {title: '敦克爾克大行動', released: 2017},
    {title: 'TENET天能', released: 2020}
] AS p
CREATE (m:Movie) SET m = p
RETURN m
```

<img src="images/MovieGraph12.png">

如此一來就可以不斷新增諾蘭所有的電影。

#### 如何同時新增所有電影並指派關係？

上面的語法解決了一直輸入 Movie 的問題，但沒有建立導演關係，那我們把前兩個做法合併起來試試：

```
MERGE (nolan:Person {name: '諾蘭'})
UNWIND [
    {title: '追隨', released: 1998},
    {title: '記憶拼圖', released: 2000},
    {title: '針鋒相對', released: 2002},
    {title: '蝙蝠俠：開戰時刻', released: 2005},
    {title: '頂尖對決', released: 2006},
    {title: '黑暗騎士', released: 2008},
    {title: '全面啟動', released: 2010},
    {title: '黑暗騎士：黎明昇起', released: 2012},
    {title: '星際效應', released: 2014},
    {title: '敦克爾克大行動', released: 2017},
    {title: 'TENET天能', released: 2020}
] AS p
CREATE (m:Movie) SET m = p
CREATE (nolan)-[:DIRECTED]->(m)
RETURN nolan, m
```

<img src="images/MovieGraph13.png">

會出現了如上的錯誤訊息，告訴我們 MERGE 和 UNWIND 之間必須有 WITH ，這是因為 Cypher 語言並不是 CRUD 讓你可以隨意排列組合操作的，必須有一定的順序，最常見的情境是：先查詢比對，後新增修改，這個是系統默認允許的隱性操作，不需要額外處理。

除了這樣的順序之外，其他都需要注意不能讓查詢和新增/刪除/修改同時操作，而是要視為各自獨立的區塊，用 WITH 來介接。正確的寫法如下

```
MERGE (nolan:Person {name: '諾蘭'})
WITH nolan
UNWIND [
    {title: '追隨', released: 1998},
    {title: '記憶拼圖', released: 2000},
    {title: '針鋒相對', released: 2002},
    {title: '蝙蝠俠：開戰時刻', released: 2005},
    {title: '頂尖對決', released: 2006},
    {title: '黑暗騎士', released: 2008},
    {title: '全面啟動', released: 2010},
    {title: '黑暗騎士：黎明昇起', released: 2012},
    {title: '星際效應', released: 2014},
    {title: '敦克爾克大行動', released: 2017},
    {title: 'TENET天能', released: 2020}
] AS p
CREATE (m:Movie) SET m = p
CREATE (nolan)-[:DIRECTED]->(m)
RETURN nolan, m
```

<img src="images/MovieGraph14.png">

#### 談到了 UNWIND，也順便看看 FOREACH 的做法，如下 (請勿重複執行)

```
MERGE (nolan:Person {name: '諾蘭'})
WITH nolan, [
    {title: '追隨', released: 1998},
    {title: '記憶拼圖', released: 2000},
    {title: '針鋒相對', released: 2002},
    {title: '蝙蝠俠：開戰時刻', released: 2005},
    {title: '頂尖對決', released: 2006},
    {title: '黑暗騎士', released: 2008},
    {title: '全面啟動', released: 2010},
    {title: '黑暗騎士：黎明昇起', released: 2012},
    {title: '星際效應', released: 2014},
    {title: '敦克爾克大行動', released: 2017},
    {title: 'TENET天能', released: 2020}
] AS movies
FOREACH (p in movies | 
    CREATE (m:Movie) SET m = p
    CREATE (nolan)-[:DIRECTED]->(m)
)
RETURN nolan
```

FOREACH 與 UNWIND 不同的是，前者是純粹程式邏輯操作，而不是把資料展開，所以最後 RETURN 時無法回傳所有電影。

<img src="images/MovieGraph15.png">

#### 更新資料

為人物增加年齡與性別兩個屬性，並更新 updated 為最新時間戳

```
UNWIND [{age:50}, {gender:'male'}] as property
MERGE (nolan:Person {name: '諾蘭'})
SET nolan += property
SET nolan.updated = timestamp()
```

<img src="images/MovieGraph16.png">

#### 更新 Relation Property

SET 可以更新 Node Property，也可以更新 Relation Property

```
MATCH (p:Person {name: 'Jessica Thompson'})-[r:REVIEWED]->(m:Movie) WHERE ID(m) = 86
SET r.rating = 85
```


#### 刪除 Node

```
MATCH (nolan:Person {name: '諾蘭'})
DELETE nolan
```

<img src="images/MovieGraph17.png">

刪除 Node 時常會遇到如下錯誤訊息，存有任何關係的 Node 是不能直接刪除的

所以請先 MATCH relations 並刪除後，再刪除 Node。另一個作法是連同 Relation 一並刪除，但這樣的操作就要特別小心

```
MATCH (nolan:Person {name: '諾蘭'})
DETACH DELETE nolan
```

<img src="images/MovieGraph18.png">

#### 根據深度搜尋

這是官方電影資料庫的範例，目的是查詢 Kevin Bacon 四度關係內的所有Node，不包含自己。

```
MATCH (bacon:Person {name:"Kevin Bacon"})-[*1..4]-(hollywood)
RETURN DISTINCT hollywood
```

<img src="images/MovieGraph19.png">

#### 如果回傳的結果要包含自己，則是從 0 開始

```
MATCH (bacon:Person {name:"Kevin Bacon"})-[*0..4]-(hollywood)
RETURN DISTINCT hollywood
```

<img src="images/MovieGraph20.png">

#### 尋找最短路徑

官方範例，目的是尋找 Kevin Bacon 到 Meg Ryan 的最短路徑，可以說是圖算領域很常見的需求

shortestPath 是 Cypher 內建函數

[*] 指的是無限制深度，這要小心使用，否則可能會導致搜索量太大造成查詢 timeout！

```
MATCH p=shortestPath(
(bacon:Person {name:"Kevin Bacon"})-[*]-(meg:Person {name:"Meg Ryan"})
)
RETURN p
```

<img src="images/MovieGraph21.png">

<a id="CreatingNodesRelationshipsGraph"></a>
## Creating Nodes and Relationships in the Graph  with Cypher

To ensure you will not add double names

`CREATE CONSTRAINT ON (p:Person) ASSERT p.name IS UNIQUE`

Add “Ann” and “Dan” to the database. Ann and Dan love each other. Ann owns a pet, “Sam”, and she drives Dan’s car. Use separate queries:

`CREATE (:Person{name:'Ann'}) – [:LOVES] ->(:Person {name:'Dan'})`

`MERGE (p:Person{name:'Ann'}) CREATE (p)-[:HAS_PET]->(:Dog {name:'Sam'})`

`MERGE (p:Person{name:'Ann'}), (dan:Person{name:'Dan'}) CREATE (p)-[:DRIVES]->(car:Car{brand:'Volvo'})<-[:OWNS]-(dan)`

<img src="images/image03.png">

When you’ve made a mistake, or you’d like to redo.. This is the most efficient way to remove all data in the database (NOTE: later on you’d have to find more specific ways to delete, since you’ll lose all data this way):

`Match (n) DETACH DELETE n`

<img src="images/image02.png">

You can also add them this way in separate queries:

`CREATE (ann:Person{name:'Ann'}) – [:LOVES] ->(dan:Person {name:'Dan'})`

`MERGE (ann:Person{name:'Ann'}) CREATE (ann)-[:HAS_PET]->(dog:Dog {name:'Sam'})`

`MERGE (ann:Person{name:'Ann'}) CREATE (ann)-[:DRIVES]->(car:Car {brand:'Volvo'})`

`MATCH (dan:Person {name:'Dan'}), ((car:Car {brand:'Volvo'})) MERGE (dan)<-[:OWNS]-(car)`

### Clean Nodes

`Match (n) DETACH DELETE n`

### Or add them all at once (as one query):

```
CREATE (ann:Person{name:'Ann'}) – [:LOVES] ->(dan:Person {name:'Dan'})
CREATE (ann)-[:HAS_PET]->(dog:Dog {name:'Sam'})
CREATE (ann)-[:DRIVES]->(car:Car {brand:'Volvo'})<-[:OWNS]-(dan)
```

### Clean Nodes

`Match (n) DETACH DELETE n`

### Adding all nodes separately, and then connect them with relationships:

```
CREATE (ann:Person{name:'Ann', age:32})
CREATE (dan:Person {name:'Dan', age:33})
CREATE (sam:Dog {name:'Sam', gender:'male'})
CREATE (car:Car {brand:'Volvo'})
```

`Match (ann:Person {name:'Ann'}),(dan:Person {name:'Dan'}) CREATE (ann)-[:LOVES]->(dan)<-[:LOVES]-(ann)`

`Match (ann:Person{name:'Ann'}),(sam:Dog{name:'Sam'}) CREATE (ann)-[:HAS_PET]->(sam)`

`Match (ann:Person{name:'Ann'}),(sam:Dog{name:'Sam'}) CREATE (sam)-[:PET_OF]->(ann)`

`Match (ann:Person{name:'Ann'}), (dan:Person{name:'Dan'}), (car:Car{brand:'Volvo'}) CREATE (ann)-[:DRIVES]->(car)<-[:OWNS]-(dan)`

### Show all entries in the graph

`MATCH (n) RETURN n`

<img src="images/image04.png">

### Now, we've had our fun. We can now create some more data and relationships:

`CREATE (jack:Person {name:'Jack', age:42, position:'manager'})`

`MATCH (jack:Person{name:'Jack'}), (dan:Person{name:'Dan'}) CREATE (jack) - [:BOSS_OF] -> (dan)`

`MATCH (dan:Person{name:'Dan'}) CREATE (dan) - [:SUPERVISES] -> (jojo:Person {name:'Jojo', age:25, position:'student'})`

###  Create some Movie, user who rates a Movie

```
CREATE (Inception:Movie{title:'Inception', director: 'Christopher Nolan'})
CREATE (DarkKnight:Movie {title:'The Dark Knight', director: 'Christopher Nolan'})
CREATE (beasts:Movie{title:'Fantastic Beasts: the crimes of Grindelwald', director: 'David Yates'})
CREATE (shawshank:Movie {title:'The Shawshank Redemption', director: 'Frank Darabont'})
CREATE (forrest:Movie {title:'Forrest Gump', director: 'Robert Zemeckis'})
CREATE (matrix:Movie {title:'The Matrix', director: 'Lana Wachowski & Lilly Wachowski'})
```

```
CREATE (Peter:User {name: 'Peter N', age:30})
CREATE (Ryan:User {name: 'Ryan A', age:20})
CREATE (Sam:User {name: 'Sam Sheldon', age:35})
CREATE (Inception)-[:RATED {rating:9}]->(Peter)
CREATE (Inception)-[:RATED {rating:8}]->(Sam)
CREATE (DarkKnight)-[:RATED {rating:6}]->(Peter)
CREATE (DarkKnight)-[:RATED {rating:7}]->(Sam)
```

### Add some more relationships:

`MATCH (sam:User {name: 'Sam Sheldon', age:35}), (dan:Person{name:'Dan', age:33}) CREATE (sam) - [:BROTHER] -> (dan)`

`MATCH (ryan:User {name: 'Ryan A', age:20}), (jack:Person{name:'Jack', age:42}) CREATE (ryan) - [:BROTHER] -> (jack)`

`MATCH (jojo:Person {name: 'Jojo'}), (peter:Person {name: 'Peter N'}) CREATE (jojo) - [:SISTER] -> (peter)`

### To make sure nodes for people are labelled the same (Person instead of User):
```
MATCH (n:User) 
WHERE ID(n) IN [10, 11, 12]
REMOVE n:User
SET n:Person
```

### Ryan is a true movie fanatic and rates all movies he's been watching lately. Also, Sam and Dan rate some movies they watched:

`MATCH (ryan:Person {name: 'Ryan A', age:20}), (inception:Movie{title:'Inception'}) CREATE (ryan) - [:RATED{rating: 9}] -> (inception)`

`MATCH (forrest:Movie{title:'Forrest Gump'}) CREATE (ryan) - [:RATED{rating: 7}] -> (forrest)`

`MATCH (beasts:Movie {title:'Fantastic Beasts: the crimes of Grindelwald'}) CREATE (ryan) - [:RATED{rating: 8}] -> (beasts)`

`MATCH (dark:Movie {title:'The Dark Knight'}) CREATE (ryan) - [:RATED{rating: 6}] -> (dark)`

`MATCH (matrix:Movie {title:'The Matrix'}) CREATE (ryan) - [:RATED{rating: 10}] -> (matrix)`

`MATCH (shawshank:Movie {title:'The Shawshank Redemption'}) CREATE (ryan) - [:RATED{rating: 9}] -> (shawshank)`

`MATCH (sam:Person {name: 'Sam Sheldon', age: 35}) CREATE (sam) - [:RATED{rating: 9}] -> (beasts)`

`MATCH (dan:Person {name: 'Dan'}) CREATE (dan) - [:RATED{rating: 9}] -> (beasts)`

### Make Jojo rate all movies with a 2:
`MATCH (jojo:Person {name: 'Jojo'}), (m:Movie) CREATE (jojo) - [:RATED {rating: 2}] -> (m)`

###  Let's show all entries in the graph:
`MATCH (n) RETURN n`

<img src="images/image05.png">

<a id="DefaultNeo4jDatasets"></a>
## Neo4j databases and graphs

<b>預存資料集</b>

除了電影資料庫之外，還有其他許多預存資料集或範例可供操作，例如經典的北風資料庫，可說是學 SQL 最佳範例。

```
:play movie-graph
```
```
:play northwind-graph
```
```
:play got
```
```
:play ukcompanies
```
```
:play listings
```
```
:play football_transfers
```

直接在 Neo4j Browser 輸入上述指令，依照指示步驟建立或匯入預存資料集。

### 社群分享的 graph

除了官方預定義的資料及之外，Neo4j最迷人的地方，就是廣大社群網友上傳分享的資料集。

#### Graph Gallery

GraphGist 是讓網友們可以自由分享資料集的地方，包含各種情境或行業例如財務、運動、社群、健康、網路、運輸 ... 你可以上去尋找有興趣的資料集做研究和練習。

想要安裝 Graph Gallery 的資料集，請在 Neo4j Desktop 選擇 Graph Apps Gallery

<img src="images/Neo4j_Desktop_Sidebar.jpg">

然後安裝 Graph Gallery，或是透過[Graph Gallery官方網站](https://install.graphapp.io) 安裝也可以。

<img src="images/Neo4j_Graph_Gallery.jpg">

安裝成功後就，從 Neo4j Desktop 點選 Graph Gallery 就可以看到畫面，之後在想要安裝的圖形資料點選 Play as Browser Guide 即可。

若你按下Install按鈕沒有反應，則可以複製套件的[連結](https://neo.jfrog.io/neo/api/npm/npm/graph-gallery)，貼在 Install 輸入方框中。

<img src="images/GraphGalleryManualInstallation.png">

<img src="images/GraphAppInstallation.png">

<img src="images/GraphExamples.png">

<a id="Neo4jSandbox"></a>

## Neo4j Sandbox

Sandbox 讓你可以連安裝 Neo4j 都不需要！直接在瀏覽器連線遠端資料庫，並線上操作。主要是讓還沒搞懂 Neo4j，不想安裝軟體的使用者可以先行體驗，所以也會有很豐富的預存資料集可供練習，畫面如下：

<img src="images/Neo4j_Sandbox.jpg">



Blank Sandbox 顧名思義就是開一個空的資料庫讓你自由操作。

Twitter 則是會要求連接你的 Twitter 帳號並建立社群網路圖。

其他資料集有興趣的話也都可以嘗試。不過 Sandbox 資料庫預設只會保留 3 天，最多延長 7 天變成 10 天，畢竟只是體驗的版本。

<img src="images/Neo4jSandBox.png">

### Neo4j Sandbox 遠端連線

Sandbox 雖然只是體驗版資料庫，但麻雀雖小五臟俱全，它是可以讓你真的做遠端連線資料庫，在 Neo4j Desktop 建立新的資料庫，選擇 Connect to Remote DBMS。

<img src="images/Neo4j_Desktop_Create_new_db.jpg">

接者回到瀏覽器你的 Sandbox 畫面，點選 Connection Details，這邊會需要的資訊是 Username、Password、Bolt URL

<img src="images/Neo4j_Sandbox_Connection_Detail.jpg">

再回到 Neo4j Desktop，依序輸入上述三項資訊，之後點選 Connect 即可。

<img src="images/Neo4j_Desktop_Remote_1.jpg">

## Summary

You should now be able to:

- Create your own node, relationships, attributes with your graph.
- Using SandBox as your play ground of Neo4j.
- Import Neo4j default datasets into Graph.
- Explore Neo4j Browser.